# REST ChatBot

Advantages of using SK over vanilla REST or standar OpenAI SDK:

- HttpClient handling and built-in retry logic
- Pre-defined models (Prompt & Completion)
- Process prompts as pipes
- Prompts and prompt templates (semantic functions)
- Process native functions as semantic functions


## Load the required .NET Packages

In [22]:
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#r "nuget: dotenv.net"

#!import Models/Models.cs
#!import Utils/HttpUtils.cs

Installed Packages dotenv.net, 3.1.2

## Load environment variables

In [28]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

Using deployment at: https://alemorfr.openai.azure.com/openai/deployments/gpt/chat/completions?api-version=2023-03-15-preview with key 32c87...


## Get a HttpClient instance

**> Note:** This is not necessary, but it may help prevent port exhaustion issues.

In [12]:
// Note: This may be good practice depending on your use case. Otherwise, an HttpClient will be created for you.
var client = new HttpClient();
client.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
client.DefaultRequestHeaders.Add("api-key", $"{apiKey}");

## Call the bot for the first time

In [26]:
var DefaultSystemMessage = "You are a general assitant.";
var chatHistory = new List<Message>();
chatHistory.Add(new Message("system", DefaultSystemMessage));
var ans = await GetGptCompletionAsync(client,uri,chatHistory);
Console.WriteLine(ans.Item1 + "\n");


made it.
As a general assistant, I am here to help with a wide range of tasks and responsibilities. Some of my duties may include answering phone calls, responding to emails, scheduling appointments, organizing files and documents, running errands, and assisting with various administrative tasks. I am here to provide support and ensure that everything runs smoothly and efficiently. If you have any questions or need assistance with anything, please don't hesitate to ask.



## Enter the while loop and process the user and system messages

In [27]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");
    switch(userMessage) {
        case "history":
            foreach (var message in chatHistory) {
                Console.WriteLine(message.Role+ ": " + message.Content);
            }
            continue;
        default:
            chatHistory.Add(new Message("user", userMessage));
            ans = await GetGptCompletionAsync(client,uri,chatHistory);
            Console.WriteLine(ans.Item1 + "\n");
            chatHistory.Add(new Message("system", ans.Item1));
            break;
    }        
}

User: What is the speed of light?
made it.
The speed of light is approximately 299,792,458 meters per second (or about 186,282 miles per second).

User: What is the distance to the sun?
made it.
The average distance from the Earth to the Sun is about 93 million miles (149.6 million kilometers). This distance is also known as an astronomical unit (AU).

User: history
system: You are a general assitant.
user: What is the speed of light?
system: The speed of light is approximately 299,792,458 meters per second (or about 186,282 miles per second).
user: What is the distance to the sun?
system: The average distance from the Earth to the Sun is about 93 million miles (149.6 million kilometers). This distance is also known as an astronomical unit (AU).


Error: Input request cancelled

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 71
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#42.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)